# Data preparation
## Import necessary libraries
In this code window, we initiate the data preparation process by importing essential libraries. The 'pandas' library is utilized for data manipulation, while 'numpy' provides support for numerical operations. Additionally, we import the 'calendar' module to work with date-related functions.


In [40]:
import pandas as pd
import numpy as np
import calendar


## Load metadata for air quality locations
Here, we retrieve metadata from a CSV file hosted online. This metadata contains information about various air quality monitoring locations. The data is fetched using the 'pd.read_csv' function from the 'pandas' library and stored in the 'meta_data' variable.


In [41]:
meta_data = pd.read_csv("https://raw.githubusercontent.com/CopernicusAtmosphere/air-quality-covid19-response/master/CAMS_AQ_LOCATIONS_V1.csv")

## Reading and Concatenating Air Quality Data
In this section, we read and compile air quality data spanning multiple years. The 'aq_data' dictionary is employed to store individual DataFrames for each year. Using a loop, data for each year is retrieved from online sources and appended to the dictionary. Finally, all the data is concatenated into a single DataFrame named 'aq_final'.


In [42]:
years = range(2015, 2024)

# Create an empty dictionary to store aq_data
aq_data = {}

# Read aq_data for each year and store in aq_data dictionary
for year in years:
    url = f"https://raw.githubusercontent.com/CopernicusAtmosphere/air-quality-covid19-response/master/cams_air_quality_analysis_{year}.csv"
    aq_data[str(year)] = pd.read_csv(url)

# Concatenate all aq_data into a single DataFrame
aq_final = pd.concat(aq_data.values(), ignore_index=True)

aq_final

,basetime,city_id,NO2,O3,PM10,PM2.5
0,2015-01-01,AQ001,25.28,30.06,41.76,19.86
1,2015-01-01,AQ002,22.67,30.05,13.58,8.95
2,2015-01-01,AQ003,7.80,63.02,7.39,4.38
3,2015-01-01,AQ004,28.66,40.01,20.09,14.16
4,2015-01-01,AQ005,14.80,35.78,50.96,28.66
...,...,...,...,...,...,...
162695,2023-11-28,AQ046,6.42,82.89,25.46,7.78
162696,2023-11-28,AQ047,20.64,23.63,14.94,12.28
162697,2023-11-28,AQ048,16.39,35.26,15.87,11.63
162698,2023-11-28,AQ049,26.41,16.89,41.56,35.22


## Data Cleaning and Feature Engineering
This code window focuses on cleaning and enhancing the air quality dataset Initial steps involve merging the data with the previously loaded metadata based on city IDs. Subsequently, rolling means for various time windows are computed to capture trends in NO2 concentrations. Monthly and yearly indices are then calculated to assess the relative changes in concentration over time. Finally, winners for both monthly and yearly challenges are determined based on the computed indices. The processed data is structured for further analysis and saved to a CSV file.


In [43]:
# Concatenate all aq_data into a single DataFrame
aq_final = pd.concat(aq_data.values(), ignore_index=True)

# Merge aq_final with meta_data on 'city_id' and 'id'
aq_final = pd.merge(aq_final, meta_data[['id', 'name', 'latitude', 'longitude']], left_on='city_id', right_on='id')

# Compute 'no2_rmean7' using rolling mean for each 'name'
aq_final['no2_rmean7'] = aq_final.groupby('name')['NO2'].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

# Compute 'no2_rmean35' using rolling mean for each 'name'
aq_final['no2_rmean35'] = aq_final.groupby('name')['NO2'].rolling(window=35, min_periods=1).mean().reset_index(0, drop=True)

# Compute 'no2_rmean1j' using rolling mean for each 'name'
aq_final['no2_rmean1j'] = aq_final.groupby('name')['NO2'].rolling(window=365, min_periods=1).mean().reset_index(0, drop=True)

# Compute 'no2_rmean2j' using rolling mean for each 'name'
aq_final['no2_rmean2j'] = aq_final.groupby('name')['NO2'].rolling(window=365*2, min_periods=1).mean().reset_index(0, drop=True)

# Compute 'no2_rmean3j' using rolling mean for each 'name'
aq_final['no2_rmean3j'] = aq_final.groupby('name')['NO2'].rolling(window=365*3, min_periods=1).mean().reset_index(0, drop=True)

# Compute 'no2_rmean_c' as the weighted average of 'no2_rmean1j', 'no2_rmean2j', and 'no2_rmean3j' for each 'name'
weights = np.array([0.2, 0.3, 0.5])

aq_final['no2_rmean_c'] = (aq_final['no2_rmean1j'] * weights[0] +
                           aq_final['no2_rmean2j'] * weights[1] +
                           aq_final['no2_rmean3j'] * weights[2])

# Convert 'basetime' column to datetime type
aq_final['basetime'] = pd.to_datetime(aq_final['basetime'])
aq_final = aq_final[aq_final['basetime'].dt.year >= 2019]

# Group the data by 'name' and the year-month of 'basetime'
grouped_m = aq_final.groupby(['name', aq_final['basetime'].dt.to_period('M')])

# Calculate the index as the value of each day divided by the value of the first day of the month, multiplied by 100
aq_final['monthly_index'] = grouped_m['no2_rmean_c'].transform(lambda x: x / x.iloc[0] * 100)

# Group the data by 'name' and the year of 'basetime'
grouped_y = aq_final.groupby(['name', aq_final['basetime'].dt.year])

# Calculate the index as the value of each day divided by the value of the first day of the year, multiplied by 100
aq_final['yearly_index'] = grouped_y['no2_rmean_c'].transform(lambda x: x / x.iloc[0] * 100)

# Rename columns
# aq_final = aq_final.rename(columns={'no2_rmean1j': '365d moving average', 'no2_rmean_c': 'Composite moving average'})
aq_final = aq_final.drop(['no2_rmean7', 'no2_rmean35', 'no2_rmean2j', 'no2_rmean3j'], axis=1)

aq_final['days_in_month'] = aq_final['basetime'].dt.month.apply(lambda x: calendar.monthrange(pd.to_datetime('today').year, x)[1])

# Determine the winner for each month
aq_final['winner_month'] = ""
for year in aq_final['basetime'].dt.year.unique():
    for month in aq_final['basetime'].dt.month.unique():
        month_data = aq_final[(aq_final['basetime'].dt.year == year) & (aq_final['basetime'].dt.month == month)]
        last_day_of_month = month_data['basetime'].dt.day.max()
        #last_day_of_month = calendar.monthrange(year, month)[1]
        days_in_month = month_data['days_in_month'].max()
        if last_day_of_month == days_in_month:
            last_day_data = month_data[month_data['basetime'] == month_data['basetime'].max()]
            if len(last_day_data) > 0:
                min_monthly_index = last_day_data['monthly_index'].min()
                winner_city_month = last_day_data[last_day_data['monthly_index'] == min_monthly_index]['name'].values[0]
                aq_final.loc[(aq_final['basetime'].dt.year == year) & (aq_final['basetime'].dt.month == month), 'winner_month'] = winner_city_month

# Determine the winner for each year
aq_final['winner_year'] = ""
for year in aq_final['basetime'].dt.year.unique():
    year_data = aq_final[aq_final['basetime'].dt.year == year]
    max_date = year_data['basetime'].max()  
    if max_date.month == 12 and max_date.day == 31:
        last_day_data = year_data[year_data['basetime'] == year_data['basetime'].max()]
        if len(last_day_data) > 0:
            min_yearly_index = last_day_data['yearly_index'].min()
            winner_city_year = last_day_data[last_day_data['yearly_index'] == min_yearly_index]['name'].values[0]
            aq_final.loc[aq_final['basetime'].dt.year == year, 'winner_year'] = winner_city_year

aq_final_csv = aq_final[['name', 'latitude', 'longitude', 'basetime', 'NO2', 'no2_rmean1j', 'no2_rmean_c', 'monthly_index', 'yearly_index', 'winner_month', 'winner_year']].copy()
aq_final_csv.columns = ['city', 'lat', 'lng', 'year', 'Raw', '365d moving average', 'Composite moving average', "Monthly Index Challenge", "Yearly Index Challenge", 'winner_month', 'winner_year']

aq_final_csv

,city,lat,lng,year,Raw,365d moving average,Composite moving average,Monthly Index Challenge,Yearly Index Challenge,winner_month,winner_year
1461,Amsterdam,52.35,4.92,2019-01-01,11.44,19.770822,20.001654,100.000000,100.000000,Sofia,Monaco
1462,Amsterdam,52.35,4.92,2019-01-02,13.17,19.757589,19.998284,99.983152,99.983152,Sofia,Monaco
1463,Amsterdam,52.35,4.92,2019-01-03,21.57,19.795589,20.004180,100.012631,100.012631,Sofia,Monaco
1464,Amsterdam,52.35,4.92,2019-01-04,21.74,19.806082,20.008093,100.032191,100.032191,Sofia,Monaco
1465,Amsterdam,52.35,4.92,2019-01-05,11.72,19.774274,19.990328,99.943377,99.943377,Sofia,Monaco
...,...,...,...,...,...,...,...,...,...,...,...
162695,Zagreb,45.80,16.00,2023-11-24,18.54,10.426329,11.656554,99.273091,92.580266,,
162696,Zagreb,45.80,16.00,2023-11-25,10.12,10.387507,11.634742,99.087330,92.407029,,
162697,Zagreb,45.80,16.00,2023-11-26,14.32,10.368986,11.625292,99.006843,92.331969,,
162698,Zagreb,45.80,16.00,2023-11-27,16.40,10.369479,11.623249,98.989449,92.315747,,


## Saving Processed Data to CSV
writing the resulting DataFrame to a CSV file.

In [44]:
aq_final_csv.to_csv('../static/data/data.csv', index=False)